# 1. 사용 패키지 호출 및 초기화

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("mypyspark").getOrCreate()

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 2. 파일 경로 변수화

In [4]:
ProductFilePath = "hdfs:///user/s21410785/01_Product.csv"
Search1FilePath = "hdfs:///user/s21410785/02_Search1.csv"
Search2FilePath = "hdfs:///user/s21410785/03_Search2.csv"
CustomFilePath = "hdfs:///user/s21410785/04_Custom.csv"
SessionFilePath = "hdfs:///user/s21410785/05_Session.csv"
MasterFilePath = "hdfs:///user/s21410785/06_Master.csv"

* 주어진 데이터를 데이터프레임으로 생성

In [5]:
Product = spark.read.csv(ProductFilePath, header='true', sep=',')
Product.createOrReplaceTempView("product")

Search1 = spark.read.csv(Search1FilePath, header='true', sep=',')
Search1.createOrReplaceTempView("search1")

Search2 = spark.read.csv(Search2FilePath, header='true', sep=',')
Search2.createOrReplaceTempView("search2")

Custom = spark.read.csv(CustomFilePath, header='true', sep=',')
Custom.createOrReplaceTempView("custom")

Session = spark.read.csv(SessionFilePath, header='true', sep=',')
Session.createOrReplaceTempView("session")

Master = spark.read.csv(MasterFilePath, header='true', sep=',')
Master.createOrReplaceTempView("master")


# 3. 데이터 탐색

### - 고객 성별, 연령층 분포 (Master.csv)

In [6]:
age_gender_group = spark.sql("""select CLNT_GENDER, CLNT_AGE, count(CLNT_ID) as COUNT
                                from custom
                                group by CLNT_GENDER,CLNT_AGE
                                order by CLNT_GENDER,CLNT_AGE""")
age_gender_group.show()

+-----------+--------+------+
|CLNT_GENDER|CLNT_AGE| COUNT|
+-----------+--------+------+
|          F|      10|  3007|
|          F|      20| 82269|
|          F|      30|271712|
|          F|      40|180398|
|          F|      50| 28881|
|          F|      60|  3784|
|          F|      70|   542|
|          F|      80|    23|
|          M|      10|   800|
|          M|      20| 13028|
|          M|      30| 41888|
|          M|      40| 35819|
|          M|      50|  7743|
|          M|      60|  1472|
|          M|      70|   304|
|          M|      80|     9|
+-----------+--------+------+



### - 구매정보의 대분류 구분 (middle_table = Product.csv + Master.csv)

In [5]:
middle_table = spark.sql("""select P.CLNT_ID,P.SESS_ID, P.PD_C, replace(P.PD_BUY_AM,',','') as PD_BUY_AM, M.CLAC1_NM
                from product as P inner join master as M
                where P.PD_C = M.PD_C
                order by CLNT_ID""")
middle_table.show()
middle_table.createOrReplaceTempView("middle")

+-------+--------+------+---------+---------+
|CLNT_ID| SESS_ID|  PD_C|PD_BUY_AM| CLAC1_NM|
+-------+--------+------+---------+---------+
|0000008|07017688|206218|    38000|     패션잡화|
|0000008|06964877|206218|    38000|     패션잡화|
|0000014|01156991|645262|    29900|     패션잡화|
|0000014|01156991|369141|    39900|     패션잡화|
|0000025|05317297|598634|    39800| 화장품/뷰티케어|
|0000025|08113243|715121|    92000|       음료|
|0000025|09506206|715121|    92000|       음료|
|0000025|06059256|715072|    47000|       음료|
|0000032|01136737|048881|    13500|     남성의류|
|0000032|01136737|038697|    30000|     남성의류|
|0000032|01136737|048881|    13500|     남성의류|
|0000032|00759569|031974|     9900|속옷/양말/홈웨어|
|0000032|00759569|031982|     9900|속옷/양말/홈웨어|
|0000033|08694850|607160|    22220|    퍼스널케어|
|0000033|09521159|607160|    22220|    퍼스널케어|
|0000033|10548225|605825|    49000| 화장품/뷰티케어|
|0000033|10300429|806941|    12500| 화장품/뷰티케어|
|0000033|08694850|613253|     7900|    세제/위생|
|0000051|09007214|777632|    22000

### - 구매정보,대분류 & 고객 특성 (finall_table = middle_table + Custom.csv)

In [6]:
final_table = spark.sql("""select M.CLNT_ID, C.CLNT_GENDER, C.CLNT_AGE, M.PD_C, M.PD_BUY_AM, M.CLAC1_NM
                        from middle as M inner join custom as C
                        where M.CLNT_ID = C.CLNT_ID
                        order by M.CLNT_ID
""")
final_table.show()
final_table.createOrReplaceTempView("final")

+-------+-----------+--------+------+---------+---------+
|CLNT_ID|CLNT_GENDER|CLNT_AGE|  PD_C|PD_BUY_AM| CLAC1_NM|
+-------+-----------+--------+------+---------+---------+
|0000008|          M|      20|206218|    38000|     패션잡화|
|0000008|          M|      20|206218|    38000|     패션잡화|
|0000014|          F|      30|369141|    39900|     패션잡화|
|0000014|          F|      30|645262|    29900|     패션잡화|
|0000032|          M|      40|031974|     9900|속옷/양말/홈웨어|
|0000032|          M|      40|038697|    30000|     남성의류|
|0000032|          M|      40|048881|    13500|     남성의류|
|0000032|          M|      40|031982|     9900|속옷/양말/홈웨어|
|0000032|          M|      40|048881|    13500|     남성의류|
|0000056|          F|      40|425529|     4500|    퍼스널케어|
|0000056|          F|      40|824386|    14800|       완구|
|0000056|          F|      40|425529|     4500|    퍼스널케어|
|0000058|          F|      30|417290|    29500|     남성의류|
|0000058|          F|      30|420859|    26000|    스포츠패션|
|0000058|     

### - final_table을 이용한 성별, 연령별 통계량

In [92]:
purchase_by_group = spark.sql("""select CLNT_GENDER, CLNT_AGE, count(PD_C) as PURC_CNT, round(avg(PD_BUY_AM),2) as AM_AVG
                                    from final
                                    group by CLNT_GENDER,CLNT_AGE
                                    order by CLNT_GENDER,CLNT_AGE""")
# PURC_CNT = 구매 건수
# AM_AVG = 성별,연령별 구매금액 평균
purchase_by_group.createOrReplaceTempView("purchase")

### - 월별 구매 건수와 총 구매금액 (Product.csv + Session.csv)

In [58]:
monthly_purchase = spark.sql("""select substr(S.SESS_DT,5,2) as SESS_MONTH, count(S.SESS_ID) as COUNT,
                    int(sum(P.PD_BUY_AM)) as PRICE
                    from product as P join session as S
                    where P.CLNT_ID = S.CLNT_ID and P.SESS_ID = S.SESS_ID
                    group by substr(S.SESS_DT,5,2) 
                    order by 1""")
# SESS_MONTH = 거래 월
# COUNT = 월별 총 구매건수
# PRICE = 월별 총 구매금액
monthly_purchase.show()

+----------+------+-------+
|SESS_MONTH| COUNT|  PRICE|
+----------+------+-------+
|        04|903765|3804295|
|        05|948295|4452588|
|        06|925036|5630790|
|        07|862453|4179100|
|        08|699428|3178120|
|        09|685929|2386900|
+----------+------+-------+



### - 일별 구매 건수

In [105]:
daily_purchase = spark.sql("""select S.SESS_DT as SESS_DAY, count(S.SESS_ID) as COUNT
                            from product as P join session as S
                            where P.CLNT_ID = S.CLNT_ID and P.SESS_ID = S.SESS_ID
                            group by 1
                            order by 1""")
# COUNT = 일별 구매 건수
daily_purchase.createOrReplaceTempView("daily_purchase")

In [100]:
daily_purchase.show()

+--------+-----+
|SESS_DAY|COUNT|
+--------+-----+
|20180401|38327|
|20180402|38025|
|20180403|37147|
|20180404|34346|
|20180405|31327|
|20180406|26383|
|20180407|22557|
|20180408|30361|
|20180409|35208|
|20180410|30140|
|20180411|34460|
|20180412|27844|
|20180413|26290|
|20180414|24573|
|20180415|28217|
|20180416|34225|
|20180417|28021|
|20180418|27068|
|20180419|26602|
|20180420|24431|
+--------+-----+
only showing top 20 rows



### - 매출액 순위에 의한 주요 상품군 (Product.csv + Custom.csv + Master.csv)

* 먼저, Product.csv에서 구매정보와 Custom.csv에서 고객ID와 고객 특성을 병합

In [9]:
prod_join_cust = spark.sql("""select P.CLNT_ID, C.CLNT_GENDER, C.CLNT_AGE, P.SESS_ID, P.HITS_SEQ, P.PD_C, P.PD_BUY_AM, P.PD_BUY_CT
                        from product as P join custom as C
                        where P.CLNT_ID = C.CLNT_ID""")
prod_join_cust.show()
prod_join_cust.createOrReplaceTempView("prod_join_cust")

+-------+-----------+--------+--------+--------+------+---------+---------+
|CLNT_ID|CLNT_GENDER|CLNT_AGE| SESS_ID|HITS_SEQ|  PD_C|PD_BUY_AM|PD_BUY_CT|
+-------+-----------+--------+--------+--------+------+---------+---------+
|0000892|          F|      40|00490462|     223|685591|   62,000|        1|
|0000892|          F|      40|00193328|      22|831782|   13,900|        1|
|0000892|          F|      40|00490462|     223|032514|   16,000|        1|
|0000892|          F|      40|00490462|     223|072183|   23,000|        1|
|0000892|          F|      40|00823450|     123|119758|   24,000|        1|
|0000892|          F|      40|00823450|     123|274600|   29,000|        1|
|0000892|          F|      40|00823450|      96|737466|   40,000|        1|
|0000892|          F|      40|00490462|     223|125584|    2,400|        1|
|0000892|          F|      40|00490462|     223|125584|    2,400|        1|
|0000892|          F|      40|00490462|     223|125580|    4,900|        1|
|0000892|   

* 앞서 만든 prod_join_cust와 Master.csv를 병합하여 대분류별 최고매출 10순위 추출

In [20]:
main_product = spark.sql("""select M.CLAC1_NM, count(P.CLNT_ID) as COUNT, round(avg(P.HITS_SEQ),1) as AVG_HITS,
                        sum(int(replace(P.PD_BUY_AM,',',''))) as SUM_PD_AM, sum(int(replace(P.PD_BUY_CT,',',''))) as SUM_PD_CT
                        from prod_join_cust as P join Master as M
                        where P.PD_C = M.PD_C
                        group by 1
                        order by 4 desc
                        limit 10""")
# COUNT = 대분류 별 구매 건수
# AVG_HITS = 대분류 별 평균 히트 수
# SUM_PD_AM = 대분류 별 총 구매금액
# SUM_PD_CT = 대분류 별 총 구매수량

In [23]:
main_product.show()

+--------+------+--------+-----------+---------+
|CLAC1_NM| COUNT|AVG_HITS|  SUM_PD_AM|SUM_PD_CT|
+--------+------+--------+-----------+---------+
|    패션잡화|372320|    83.5|36945853350|   385716|
|    여성의류|547506|   103.5|27804838980|   555161|
|   스포츠패션|444117|    92.3|22979026310|   459423|
|화장품/뷰티케어|461686|    71.2|21641850740|   512522|
|    남성의류|411407|   103.6|13663131150|   429489|
|      가구| 33990|    67.5| 9147327640|    41719|
|   유아동의류|282643|   117.2| 8221223940|   288458|
| 생활/주방가전| 32957|    70.1| 7800062020|    34251|
|    건강식품| 95194|    68.0| 6389955380|   122867|
| 냉장/세탁가전|  4834|    72.5| 5198983240|     4861|
+--------+------+--------+-----------+---------+



* 총 구매금액 순위 대분류 TOP10 리스트

In [24]:
main_product_list = main_product.toPandas()["CLAC1_NM"]
main_product_list

0        패션잡화
1        여성의류
2       스포츠패션
3    화장품/뷰티케어
4        남성의류
5          가구
6       유아동의류
7     생활/주방가전
8        건강식품
9     냉장/세탁가전
Name: CLAC1_NM, dtype: object

### - 고객 특성별 주요상품군 구매현황

In [29]:
main_product_by_gender = spark.sql("""select M.CLAC1_NM, P.CLNT_GENDER, P.CLNT_AGE, count(P.CLNT_ID) as COUNT, round(avg(P.HITS_SEQ),1) as AVG_HITS,
                                sum(int(replace(P.PD_BUY_AM,',',''))) as SUM_PD_AM, sum(int(replace(P.PD_BUY_CT,',',''))) as SUM_PD_CT
                                from prod_join_cust as P join Master as M
                                where P.PD_C = M.PD_C
                                group by 1,2,3
                                order by 5 desc""").toPandas()


In [31]:
main_product_by_gender = main_product_by_gender.set_index("CLAC1_NM")
# 전체 대분류 중에서 TOP10 대분류만 추출하기 위해 대분류를 인덱스로 설정

In [32]:
main_product_by_gender

,CLNT_GENDER,CLNT_AGE,COUNT,AVG_HITS,SUM_PD_AM,SUM_PD_CT
CLAC1_NM,,,,,,
냉동식품,M,10,1,273.0,9900,1
과일,M,10,1,273.0,9900,1
시즌스포츠,M,80,5,236.4,104900,5
패션잡화,F,60,6155,186.5,1233550530,6222
세제/위생,F,10,22,155.4,414150,26
헬스/피트니스,M,70,3,139.7,24100,3
침구/수예,M,10,4,135.8,63200,4
침구/수예,F,10,27,132.2,975500,32
유아동의류,M,70,124,123.6,4850850,124


* TOP10에 해당하는 대분류만 추출하여 p_main_product로 저장
* 대분류 데이터를 테이블에 열로 삽입
* 그후 테이블 데이터 내용 정리 및 정렬(대분류 > 고객성별 > 고객 연령 순)

In [42]:
p_main_product = main_product_by_gender.loc[main_product_list,:]
p_main_product["CLAC1_NM"] = p_main_product.index.values

def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

p_main_product = change_column_order(p_main_product, 'CLNT_GENDER', 0)
p_main_product = change_column_order(p_main_product, 'CLAC1_NM', 0)

p_main_product = p_main_product.reset_index(drop = True)

p_main_product = p_main_product.sort_values(["CLAC1_NM","CLNT_GENDER","CLNT_AGE"])
p_main_product

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,CLAC1_NM,CLNT_GENDER,CLNT_AGE,COUNT,AVG_HITS,SUM_PD_AM,SUM_PD_CT
91,가구,F,10,15,43.3,2035300,17
88,가구,F,20,2495,63.4,520178700,2741
87,가구,F,30,13531,64.8,3349369440,16331
82,가구,F,40,10151,70.9,2765260100,12704
80,가구,F,50,1961,73.3,644290750,2577
81,가구,F,60,232,72.6,95748970,265
89,가구,F,70,35,63.2,6547250,40
79,가구,F,80,4,90.0,141000,6
92,가구,M,10,5,42.8,621850,5
85,가구,M,20,476,67.6,115115280,559


# 4. 데이터 외부로 추출

In [7]:
p_age_gender_group = age_gender_group.toPandas()
p_age_gender_group.to_csv("age_gender_group.csv",mode = 'w')

In [70]:
p_monthly_purchase = monthly_purchase.toPandas()
p_monthly_purchase.to_csv("Monthly_Purchase.csv",mode="w")

In [114]:
p_daily_purchase = daily_purchase.toPandas()
p_daily_purchase["DAYS"] = pd.date_range("20180401",periods = len(p_daily_purchase["SESS_DAY"]))
p_daily_purchase.to_csv("Daily_Purchase.csv",mode='w')

In [27]:
p_main_product = main_product.toPandas()
p_main_product.to_csv("main_product.csv")

In [41]:
p_main_product.to_csv("Main_Product_Gender_Age.csv",mode='w')